# Channel Break Out Detection

## Import Data

In [97]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import stats

In [98]:
df = pd.read_csv("EURUSD_Candlestick_1_D_BID_04.05.2003-21.01.2023.csv")

In [99]:
backcandles = 45

## Detect Pivots/Fractals

In [100]:
def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot high, 2 if pivot low, 3 if both and 0 default
    """
    if candle-window < 0 or candle+window >= len(df):
        return 0
    
    pivotHigh = 1
    pivotLow = 2
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle].Low > df.iloc[i].Low:
            pivotLow=0
        if df.iloc[candle].High < df.iloc[i].High:
            pivotHigh=0
    if (pivotHigh and pivotLow):
        return 3
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0

In [101]:
window=3
df['isPivot'] = df.apply(lambda x: isPivot(x.name,window), axis=1)

## Detect Price Channel

In [102]:
def collect_channel(candle, backcandles, window):
    localdf = df[candle-backcandles-window:candle-window]
    #localdf['isPivot'] = localdf.apply(lambda x: isPivot(x.name,window), axis=1)
    highs = localdf[localdf['isPivot']==1].High.values
    idxhighs = localdf[localdf['isPivot']==1].High.index
    lows = localdf[localdf['isPivot']==2].Low.values
    idxlows = localdf[localdf['isPivot']==2].Low.index
    
    if len(lows)>=3 and len(highs)>=3:
        sl_lows, interc_lows, r_value_l, _, _ = stats.linregress(idxlows,lows)
        sl_highs, interc_highs, r_value_h, _, _ = stats.linregress(idxhighs,highs)
    
        return(sl_lows, interc_lows, sl_highs, interc_highs, r_value_l**2, r_value_h**2)
    else:
        return(0,0,0,0,0,0)

In [103]:
df['Channel'] = [collect_channel(candle, backcandles, window) for candle in df.index]

## Detect Break Out

In [104]:
def isBreakOut(candle, backcandles, window):
    if (candle-backcandles-window)<0:
        return 0
    
    sl_lows, interc_lows, sl_highs, interc_highs, r_sq_l, r_sq_h = df.iloc[candle].Channel
    
    prev_idx = candle-1
    prev_high = df.iloc[candle-1].High
    prev_low = df.iloc[candle-1].Low
    prev_close = df.iloc[candle-1].Close
    
    curr_idx = candle
    curr_high = df.iloc[candle].High
    curr_low = df.iloc[candle].Low
    curr_close = df.iloc[candle].Close
    curr_open = df.iloc[candle].Open

    if ( prev_high > (sl_lows*prev_idx + interc_lows) and
        prev_close < (sl_lows*prev_idx + interc_lows) and
        curr_open < (sl_lows*curr_idx + interc_lows) and
        curr_close < (sl_lows*prev_idx + interc_lows)): #and r_sq_l > 0.9
        return 1
    
    elif ( prev_low < (sl_highs*prev_idx + interc_highs) and
        prev_close > (sl_highs*prev_idx + interc_highs) and
        curr_open > (sl_highs*curr_idx + interc_highs) and
        curr_close > (sl_highs*prev_idx + interc_highs)): #and r_sq_h > 0.9
        return 2
    
    else:
        return 0

In [105]:
df["isBreakOut"] = [isBreakOut(candle, backcandles, window) for candle in df.index]

In [106]:
def SIGNAL():
    return df.isBreakOut

In [124]:
from backtesting import Strategy
from backtesting import Backtest
import backtesting

class BreakOut(Strategy):
    initsize = 0.1
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 1.2

        if self.signal1==2 and len(self.trades)==0:   
            sl1 = self.data.Low[-2]
            tp1 = self.data.Close[-1] + abs(self.data.Close[-1]-sl1)*TPSLRatio
            #tp2 = self.data.Close[-1] + abs(self.data.Close[-1]-sl1)*TPSLRatio/3
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
            #self.buy(sl=sl1, tp=tp2, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.High[-2]
            tp1 = self.data.Close[-1] - abs(sl1-self.data.Close[-1])*TPSLRatio
            #tp2 = self.data.Close[-1] - abs(sl1-self.data.Close[-1])*TPSLRatio/3
            self.sell(sl=sl1, tp=tp1, size=self.mysize)
            #self.sell(sl=sl1, tp=tp2, size=self.mysize)

bt = Backtest(df, BreakOut, cash=1000, margin=1/50, commission=.000)
stat = bt.run()
stat

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_4460\1909120538.py:30: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, BreakOut, cash=1000, margin=1/50, commission=.000)


Start                                     0.0
End                                    7202.0
Duration                               7202.0
Exposure Time [%]                   46.036374
Equity Final [$]                  2400.275022
Equity Peak [$]                   2400.275022
Return [%]                         140.027502
Buy & Hold Return [%]               -3.227273
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -77.336678
Avg. Drawdown [%]                   -8.597757
Max. Drawdown Duration                 4228.0
Avg. Drawdown Duration             332.190476
# Trades                                338.0
Win Rate [%]                        47.633136
Best Trade [%]                       4.849234
Worst Trade [%]                     -4.553677
Avg. Trade [%]                    

In [110]:
bt.plot()

Row(id='17848', ...)

In [123]:
df =df.reset_index()

In [122]:
df = df.drop("Gmt time", axis=1)